In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [4]:
label = train['class']

In [9]:
api_embedding_file_path = 'word_embedding_300dim_new.txt'
embedding_dim = 300
max_nb_api = 20000
max_sequence_length_api = 2000

In [5]:
data = pd.DataFrame()

In [10]:
char_embedding_index = {}
f = open('word_embedding_300dim_new.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

16053it [00:01, 9306.08it/s]


In [13]:
all_word = list(train['article'].values) + list(test['article'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

204554

In [14]:
tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['char_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

In [15]:
api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

Found 16052 unique tokens


In [16]:
for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

100%|██████████| 16052/16052 [00:00<00:00, 198183.12it/s]


In [17]:
data['id'] = train['id']

In [18]:
train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

In [19]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [20]:
def get_input(df):
    
    _input = [np.array(df.char_seq.values.tolist())]
    
    
    return _input 

In [21]:
def CNN1():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    char_cnn = api_embedding_layer(char_input)
    
    kernel_sizes = [1, 2, 3, 4, 5, 6]
    pooled_word_q1 = []
    pooled_word_q2 = []
    
    for kernel in kernel_sizes:
            
        conv_word_q1 = Conv1D(filters=300,
                          kernel_size=kernel,
                          padding='same',
                          strides=1, 
                          kernel_initializer='he_uniform',
                          activation='relu')(char_cnn)

        pool_word_q1 = MaxPooling1D(pool_size = max_sequence_length_api)(conv_word_q1)
        pool_word_q2 = AveragePooling1D(pool_size = max_sequence_length_api)(conv_word_q1)
        
        pooled_word_q1.append(pool_word_q1)
        pooled_word_q2.append(pool_word_q2)
        
    merged_cnn_word_q1 = Concatenate(axis=-1)(pooled_word_q1)
    merged_cnn_word_q2 = Concatenate(axis=-1)(pooled_word_q2)
    
    flatten_cnn_word_q1 = Flatten()(merged_cnn_word_q1)
    flatten_cnn_word_q2 = Flatten()(merged_cnn_word_q2)
    
    merged = concatenate([flatten_cnn_word_q1, flatten_cnn_word_q2])
    merged = Dropout(0.2)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(300, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [22]:
label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [23]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
#     bst_model_path = '5folds_' + 'dae_nn_v1_' + str(i) + '.hdf5'
#     early_stopping =EarlyStopping(monitor='val_loss', patience=10)
#     model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
#     callbacks = [
#             early_stopping,
#             model_checkpoint
#         ]
    model = CNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=6, batch_size=BATCH_SIZE, shuffle=True,)#callbacks=callbacks
    
#     model = NN()
#     model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/6
92040/92040 [==============================] - 598s 6ms/step - loss: 0.6961 - acc: 0.7891 - val_loss: 0.8310 - val_acc: 0.7541
Epoch 4/6
92040/92040 [==============================] - 598s 6ms/step - loss: 0.5801 - acc: 0.8194 - val_loss: 0.8544 - val_acc: 0.7564
Epoch 5/6
92040/92040 [==============================] - 598s 6ms/step - loss: 0.4731 - acc: 0.8505 - val_loss: 0.9055 - val_acc: 0.7561
Epoch 6/6
102277/102277 [==============================] - 152s 1ms/step
*******1*******
Train on 92043 samples, validate on 10234 samples
Epoch 1/6
92043/92043 [==============================] - 600s 7ms/step - loss: 1.1463 - acc: 0.6676 - val_loss: 0.9146 - val_acc: 0.7368
Epoch 2/6
92043/92043 [==============================] - 599s 7ms/step - loss: 0.8320 - acc: 0.7537 - val_loss: 0.8782 - val_acc: 0.7462
Epoch 3/6
92043/92043 [==============================] - 599s 7ms/step - loss: 0.6996 - acc: 0.7881 - val_loss

In [29]:
meta_train.columns = ['meta_cnn_char_v1_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [30]:
meta_train.to_csv('/home/libo/daguan/stack/train_meta_cnn_char_v1.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack/test_meta_cnn_char_v1.csv', index = None ,encoding = 'utf-8')

In [31]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 2000, 300)    4816200     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 2000, 300)    90300       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 2000, 300)    180300      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [43]:
gap = 100

In [44]:
def split_sentence(data, k):
    return data[:, k*gap:(k+1)*gap, :]

In [53]:
def CNN1():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    char_cnn = api_embedding_layer(char_input)
    
    pooled_word_q1 = []
    
    for k_ in range(20):
        temp = Lambda(split_sentence, name = 'split_sentence'+str(k_), arguments={'k':k_})(char_cnn)

        kernel_sizes = [1,2,3,4,5,6,7]

        for kernel in kernel_sizes:

            conv_word_q1 = Conv1D(filters=300,
                              kernel_size=kernel,
                              padding='same',
                              strides=1, 
                              kernel_initializer='he_uniform',
                              activation='relu')(temp)

            pool_word_q1 = MaxPooling1D(pool_size = 100)(conv_word_q1)

            pooled_word_q1.append(pool_word_q1)

    merged_cnn_word_q1 = Concatenate(axis=-1)(pooled_word_q1)

    flatten_cnn_word_q1 = Flatten()(merged_cnn_word_q1)
    
    merged = Dropout(0.2)(flatten_cnn_word_q1)
    merged = BatchNormalization()(merged)
    merged = Dense(300, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model


In [58]:
meta_test = pd.DataFrame()

In [ ]:
%%time
logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'stack_cnn_char_v1_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = CNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=30, batch_size=BATCH_SIZE, shuffle=True,callbacks=callbacks)#callbacks=callbacks
    
    model = CNN1()
    model.load_weights(bst_model_path)
    
    prob = model.predict(X_test,batch_size=64,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/30
92040/92040 [==============================] - 989s 11ms/step - loss: 1.2921 - acc: 0.6281 - val_loss: 0.9760 - val_acc: 0.7153
Epoch 2/30
92040/92040 [==============================] - 800s 9ms/step - loss: 0.7106 - acc: 0.7916 - val_loss: 0.9816 - val_acc: 0.7244
Epoch 3/30
92040/92040 [==============================] - 717s 8ms/step - loss: 0.3684 - acc: 0.8940 - val_loss: 1.0647 - val_acc: 0.7311
Epoch 4/30
92040/92040 [==============================] - 2258s 25ms/step - loss: 0.2363 - acc: 0.9362 - val_loss: 1.1801 - val_acc: 0.7324
Epoch 5/30
92040/92040 [==============================] - 715s 8ms/step - loss: 0.1785 - acc: 0.9541 - val_loss: 1.2238 - val_acc: 0.7348
Epoch 6/30
92056/92056 [==============================] - 718s 8ms/step - loss: 0.1516 - acc: 0.9601 - val_loss: 1.2733 - val_acc: 0.7365
Epoch 7/30
92057/92057 [==============================] - 721s 8ms/step - loss: 0.3655 - acc: 0.8955 - 